In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import requests
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

## gpd.read_file on GEOjson
crashes = gpd.read_file('Traffic Crashes - Crashes.geojson')

In [ ]:
endpoint_crashes = 'https://data.cityofchicago.org/resource/85ca-t3if.geojson?$select=crash_record_id,crash_date,location&$limit=650000'
res = requests.get(endpoint_crashes)
res = res.json()
res

In [ ]:
# this is the line that works when you don't do any select= in the endpoint
crashes_df = gpd.GeoDataFrame.from_features(res['features'])
#crashes_df = pd.DataFrame(res)
#gpd.GeoDataFrame(res)
#crashes_df.columns

In [ ]:
crashes_df.head()

In [ ]:
crashes_df.info()

In [ ]:
type(crashes_df)

In [ ]:
#filter out crashes where there was debris on roadway
crashes_df = crashes_df.loc[crashes_df['road_defect'] != 'DEBRIS ON ROADWAY'] 

In [ ]:
crashes_df.shape

In [ ]:
#filter out crashes where the traffic control device was functioning improperly or not functioning
device_not_working = ['FUNCTIONING IMPROPERLY','NOT FUNCTIONING']
crashes_df = crashes_df.loc[~crashes_df['device_condition'].isin(device_not_working)]
#crashes.loc[crashes['crash_record_id'].isin(crash_id_peds)]

In [ ]:
## read_csv on people.csv
people_df = pd.read_csv('Traffic_Crashes_-_People.csv', usecols = ['PERSON_ID', 'PERSON_TYPE', 'CRASH_RECORD_ID', 'INJURY_CLASSIFICATION','DRIVER_ACTION', 'CRASH_DATE'])

In [ ]:
peds_df = people_df.loc[people_df['PERSON_TYPE'] == 'PEDESTRIAN']
crash_id_peds = peds_df['CRASH_RECORD_ID'].tolist()
crash_id_peds

In [ ]:
ped_crashes_df = crashes_df.loc[crashes_df['crash_record_id'].isin(crash_id_peds)].reset_index(drop=True)

In [ ]:
ped_crashes_df

In [ ]:
ped_crashes_df.loc[0, 'geometry'].x

In [ ]:
print(ped_crashes_df.crs)

In [ ]:
ped_crashes_df.loc[0, 'geometry']
print(ped_crashes_df.loc[0, 'geometry'])

In [ ]:
#there is one row in there with POINT of (0,0) and it throws off plotting the map, so get rid of it
ped_crashes_df = ped_crashes_df[ped_crashes_df['geometry'].x != 0]

In [ ]:
#filter out null geometries
ped_crashes_df = ped_crashes_df[ped_crashes_df['geometry'].notna()]

In [ ]:
ped_crashes_df.plot();

In [ ]:
area_center = [41.881288,-87.686729]
peds_cluster_map = folium.Map(location =  area_center, zoom_start = 11)
marker_cluster = MarkerCluster().add_to(peds_cluster_map)
folium.GeoJson(ped_crashes_df).add_to(peds_cluster_map)

for row_index, row_values in ped_crashes_df.iterrows():
    loc = (row_values['geometry'].y, row_values['geometry'].x)
    
    marker = folium.Marker(location = loc)
    
    marker.add_to(marker_cluster)


peds_cluster_map

In [ ]:
#split POINT geometries into separate lat and long columns
ped_crashes_df['lon'] = ped_crashes_df.geometry.apply(lambda p: p.x)
ped_crashes_df['lat'] = ped_crashes_df.geometry.apply(lambda p: p.y)

In [ ]:
ped_crashes_df

In [ ]:
area_center = [41.881288,-87.686729]
cluster_only_map = folium.Map(location =  area_center, zoom_start = 11)
folium.GeoJson(ped_crashes_df).add_to(cluster_only_map)

locations = ped_crashes_df[['lat', 'lon']].values.tolist()
cluster_only_map.add_child(
    FastMarkerCluster(locations)
)

cluster_only_map

map_37207 = folium.Map(location =  area_center, zoom_start = 12)

folium.GeoJson(polygon37207).add_to(map_37207)

#create a list of locations and pass them to FastMarkerCluster()
#FastMarkerCluster gives us locations only, no detail when you hover over
locations = stops_in_37207[['lat', 'lng']].values.tolist()
map_37207.add_child(
    FastMarkerCluster(locations)
)



#display our map
map_37207

In [ ]:
peds_map